In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.metrics import f1_score
from ucimlrepo import fetch_ucirepo 
import mlrose_hiive as mr
import time 


In [ ]:
# fetch dataset 
bank_marketing = fetch_ucirepo(id=222) 

# data (as pandas dataframes) 
x_bank = bank_marketing.data.features.copy() 
y_bank = bank_marketing.data.targets.copy() 

In [ ]:
bank_data = pd.concat([x_bank, y_bank], axis=1)

In [ ]:
for column in ['job','contact','poutcome']:
    mode_value = bank_data[column].mode()[0]
    bank_data.loc[:, column] = bank_data[column].fillna(mode_value)

In [ ]:
bank_data = pd.get_dummies(bank_data, columns=['job', 'marital','education','month','poutcome'], drop_first=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
binary_columns = ['default','housing','loan','contact','y']
label = LabelEncoder()
for col in binary_columns:
    bank_data[col] = label.fit_transform(bank_data[col])

In [ ]:
X_bank = bank_data.drop('y', axis=1)
y_bank = bank_data['y']

In [ ]:
X_bank_train, X_bank_test, y_bank_train, y_bank_test = train_test_split(X_bank, y_bank, test_size=0.20, random_state=43)

In [ ]:
scaler = StandardScaler()

X_bank_train = scaler.fit_transform(X_bank_train)
X_bank_test = scaler.transform(X_bank_test)

In [ ]:
from imblearn.over_sampling import SMOTE

sampling = SMOTE(random_state=43)
X_bank_train, y_bank_train = sampling.fit_resample(X_bank_train, y_bank_train)

In [ ]:
# New function for discretizing weights
from sklearn.decomposition import PCA


def discretize_weights(weights, bins=10):
    """Discretize weights into a specified number of bins."""
    return np.digitize(weights, bins=np.linspace(weights.min(), weights.max(), bins))

def compare_continuous_discrete(nn, X, y):
    """Compare performance of continuous and discretized weights."""
    continuous_pred = nn.predict(X)
    continuous_accuracy = f1_score(y, continuous_pred)
    
    # Get the weights from the neural network
    weights = nn.fitted_weights
    
    # Discretize the weights
    discrete_weights = discretize_weights(weights)
    
    # Create a new neural network with discretized weights
    discrete_nn = mr.NeuralNetwork(
        hidden_nodes=nn.hidden_nodes,
        activation=nn.activation,
        algorithm=nn.algorithm,
        max_iters=1,
        bias=nn.bias,
        is_classifier=nn.is_classifier,
        learning_rate=nn.learning_rate,
        early_stopping=nn.early_stopping,
        clip_max=nn.clip_max,
        max_attempts=nn.max_attempts,
        random_state=nn.random_state
    )
    discrete_nn.fitted_weights = discrete_weights
    
    # Fit the discrete neural network to initialize internal structures
    discrete_nn.fit(X, y)
    
    discrete_pred = discrete_nn.predict(X)
    discrete_accuracy = f1_score(y, discrete_pred)
    
    return continuous_accuracy, discrete_accuracy

def plot_weight_distribution(nn, algo_name):
    weights = nn.fitted_weights
    plt.figure(figsize=(10, 6))
    plt.hist(weights, bins=50, edgecolor='black')
    plt.title(f'Weight Distribution for {algo_name}')
    plt.xlabel('Weight Value')
    plt.ylabel('Frequency')
    plt.show()

# Function to plot decision boundaries
def plot_decision_boundary(nn, X, y, algo_name):
    # Reduce to 2D using PCA
    pca = PCA(n_components=2)
    X_2d = pca.fit_transform(X)
    
    # Create a mesh grid
    x_min, x_max = X_2d[:, 0].min() - 1, X_2d[:, 0].max() + 1
    y_min, y_max = X_2d[:, 1].min() - 1, X_2d[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                         np.arange(y_min, y_max, 0.1))
    
    # Predict for each point in the mesh
    Z = nn.predict(pca.inverse_transform(np.c_[xx.ravel(), yy.ravel()]))
    Z = Z.reshape(xx.shape)
    
    # Plot the decision boundary
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, alpha=0.4)
    plt.scatter(X_2d[:, 0], X_2d[:, 1], c=y, alpha=0.8)
    plt.title(f'Decision Boundary for {algo_name}')
    plt.xlabel('PCA Feature 1')
    plt.ylabel('PCA Feature 2')
    plt.show()

from sklearn.model_selection import learning_curve
from sklearn.base import BaseEstimator, ClassifierMixin

class NeuralNetworkWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, **kwargs):
        self.nn = mr.NeuralNetwork(**kwargs)
    
    def fit(self, X, y):
        self.nn.fit(X, y)
        return self
    
    def predict(self, X):
        return self.nn.predict(X)
    
# Function to plot learning curves
def plot_learning_curves(X, y, algorithms):
    plt.figure(figsize=(15, 5))
    
    for i, algo in enumerate(algorithms):
        nn_wrapper = NeuralNetworkWrapper(
            hidden_nodes=[2], activation='relu', algorithm=algo, max_iters=1000,
            bias=True, is_classifier=True, learning_rate=0.1, early_stopping=True, 
            clip_max=5, max_attempts=100, random_state=43
        )
        
        train_sizes, train_scores, test_scores = learning_curve(
            nn_wrapper, X, y, cv=5, n_jobs=-1, 
            train_sizes=np.linspace(0.1, 1.0, 10),
            scoring='f1'
        )
        
        train_mean = np.mean(train_scores, axis=1)
        train_std = np.std(train_scores, axis=1)
        test_mean = np.mean(test_scores, axis=1)
        test_std = np.std(test_scores, axis=1)
        
        plt.subplot(1, 3, i+1)
        plt.plot(train_sizes, train_mean, 'o-', color='r', label='Training score')
        plt.plot(train_sizes, test_mean, 'o-', color='g', label='Cross-validation score')
        plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='r')
        plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1, color='g')
        plt.xlabel('Training examples')
        plt.ylabel('F1 Score')
        plt.title(f'Learning Curve for {algo}')
        plt.legend(loc='best')
    
    plt.tight_layout()
    plt.show()

In [ ]:
activation_functions = ['relu', 'sigmoid', 'tanh']
algorithms = ['random_hill_climb', 'simulated_annealing', 'genetic_alg']

neural_nets = {
    f"{algo}_{activ}": mr.NeuralNetwork(
        hidden_nodes=[2], activation=activ, algorithm=algo, max_iters=1000,
        bias=True, is_classifier=True, learning_rate=0.1, early_stopping=True, 
        clip_max=5, max_attempts=100, random_state=43, curve=True
    )
    for algo in algorithms
    for activ in activation_functions
}

In [ ]:
# Train and evaluate each neural network
results = {}
for name, nn in neural_nets.items():
    nn.fit(X_bank_train, y_bank_train)
    y_test_pred = nn.predict(X_bank_test)
    accuracy = f1_score(y_bank_test, y_test_pred)
    results[name] = accuracy

    # Compare continuous and discrete weights
    cont_acc, disc_acc = compare_continuous_discrete(nn, X_bank_test, y_bank_test)
    print(f"{name} - Continuous accuracy: {cont_acc:.4f}, Discrete accuracy: {disc_acc:.4f}")
    
    # Plot weight distribution
    plot_weight_distribution(nn, name)
    
    # Plot decision boundary
    plot_decision_boundary(nn, X_bank_test, y_bank_test, name)

In [ ]:
# Call the function to plot learning curves
algorithms = ['random_hill_climb', 'simulated_annealing', 'genetic_alg']
plot_learning_curves(X_bank_train, y_bank_train, algorithms)